In [6]:
f = open('insulin_pep.uniprot.fa','r')
f_traceback = open('traceback.txt','w')

seq_list = {}

for line in f:
    if line.startswith('>'):
        seq_h = line.strip().lstrip('>').split()[0]
        seq_list[seq_h] = ''
    else:
        seq_list[seq_h] += line.strip()
f.close

#print(seq_list)
#for key, item in seq_list.items():
#    print(key)
    
    
f_blo = open('BLOSUM62.txt','r')
amino = []
point = []
amino_table = {}

for line in f_blo:
    if line.startswith('#'):
        continue
    elif line.startswith(' '):
        amino = line.strip().split()
    else:
        amino_x = {}
        point = line.strip().split()

        for i in range(1,len(amino)+1):
            amino_x[amino[i-1]] = point[i]
            
        amino_table[point[0]] = amino_x
    
f_blo.close

    
    
species = []

for i in seq_list:
    species.append(i)
    
#print(species)

#experiment
#seq_list[species[1]] = 'TTCATA'
#seq_list[species[0]] = 'TGCTCGTA'
for n in range(1,len(species)):
    print('vs %s'%species[n])
    #print(seq_list[species[0]])
    #print(seq_list[species[n]])

    x_len = len(seq_list[species[0]])
    y_len = len(seq_list[species[n]])
    
    gap_penalty = 10
    ext_penalty = 0.5
    
    matrix = [[0 for i in range(x_len + 1)] for j in range(y_len + 1)]
    for i in range(x_len + 1):
        matrix[0][i] = -6 * i
    for j in range(y_len + 1):
        matrix[j][0] = -6 * j
    #print(matrix)


    traceback = [[1 for i in range(x_len + 1)] for j in range(y_len + 1)]

    for i in range(1,x_len + 1):
        for j in range(1,y_len + 1):
            score = int(amino_table[seq_list[species[0]][i-1]][seq_list[species[n]][j-1]])
            case_1 = matrix[j-1][i-1] + score
            if traceback[j][i-1] % 3 == 0:
                case_2 = matrix[j][i-1] - ext_penalty
            else:
                case_2 = matrix[j][i-1] - gap_penalty
            if traceback[j][i-1] % 5 == 0:
                case_3 = matrix[j-1][i] - ext_penalty
            else:
                case_3 = matrix[j-1][i] - gap_penalty
            matrix[j][i] = max(case_1,case_2,case_3)
            if matrix[j][i] == case_1:
                traceback[j][i] *= 2
            if matrix[j][i] == case_2:
                traceback[j][i] *= 3
            if matrix[j][i] == case_3:
                traceback[j][i] *= 5

    
    for i in range(y_len + 1):
        series = str(traceback[i]).split(',')
        series2 = ''
        for j in series:
            series2 = series2 + '%s\t'%j
        f_traceback.write(series2 + '\n')
    #    print(traceback[i])

    x_tracker, y_tracker = x_len, y_len
    #print(x_tracker, y_tracker)
    x_compare = ''
    y_compare = ''
    gap_count = 0
    id_count = 0
    
    while x_tracker + y_tracker > 0:
        if traceback[y_tracker][x_tracker] % 2 == 0:
            x_compare = seq_list[species[0]][x_tracker - 1] + x_compare
            y_compare = seq_list[species[n]][y_tracker - 1] + y_compare
            if seq_list[species[0]][x_tracker - 1] == seq_list[species[n]][y_tracker - 1]:
                id_count += 1
            x_tracker -= 1
            y_tracker -= 1
        elif traceback[y_tracker][x_tracker] % 3 == 0:
            x_compare = seq_list[species[0]][x_tracker - 1] + x_compare
            y_compare = '-' + y_compare
            x_tracker -= 1
            gap_count += 1
        elif traceback[y_tracker][x_tracker] % 5 == 0:
            x_compare = '-' + x_compare
            y_compare = seq_list[species[n]][y_tracker - 1] + y_compare
            y_tracker -= 1
            gap_count += 1
    
    seq_compare = ''
    seq_eq = '|'
    seq_neq = '.'
    seq_gap = ' '
    seq_num = 0
    while seq_num < x_len:
        if x_compare[seq_num] == '-' or y_compare[seq_num] == '-':
            seq_compare = seq_compare + seq_gap
            seq_num += 1
        elif x_compare[seq_num] != y_compare[seq_num]:
            seq_compare = seq_compare + seq_neq
            seq_num += 1
        else:
            seq_compare = seq_compare + seq_eq
            seq_num += 1
    
    id_pos = seq_compare.count('|')/len(x_compare)*100
    gap_pos = seq_compare.count(' ')/len(x_compare)*100
    print(x_compare, '   %s'%x_len)
    print(seq_compare)
    print(y_compare, '   %s'%y_len)
    print('Length :\t', len(x_compare))
    print('Identity :\t %s (%s%%)'%(seq_compare.count('|'),round(id_pos, 1)))
    print('Gaps :\t\t %s (%s%%)'%(seq_compare.count(' '),round(gap_pos, 1)))
    print('Score is %s.' %matrix[y_len][x_len], '\n')
    f_traceback.write('\n')
f_traceback.close()

vs sp|P01315|INS_PIG
MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQVGQVELGGGPGAGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN    110
||||.||||||||||||.|.||.||||||||||||||||||||||||||||||.|||||..|.|.||| || |.|.||.|||||..||||||||||||||||||||||||
MALWTRLLPLLALLALWAPAPAQAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREAENPQAGAVEL-GG-GLGGLQALALEGPPQKRGIVEQCCTSICSLYQLENYCN    108
Length :	 110
Identity :	 94 (85.5%)
Gaps :		 2 (1.8%)
Score is 484.5. 

vs sp|P01317|INS_BOVIN
MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQVGQVELGGGPGAGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN    110
||||.||.|||||||||.|.||.||||||||||||||||||||||||||||||.|||.|..|||..||.||||||     .|||..||||||||||.|.|||||||||||
MALWTRLRPLLALLALWPPPPARAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREVEGPQVGALELAGGPGAG-----GLEGPPQKRGIVEQCCASVCSLYQLENYCN    105
Length :	 110
Identity :	 88 (80.0%)
Gaps :		 5 (4.5%)
Score is 461.0. 

vs sp|P01325|INS1_MOUSE
MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQVGQVELGGGPGAGSLQPLALEGSLQKRGIV